In [ ]:
%cd /content
!git clone https://github.com/rica-team/rica-server.git
%cd /content/rica-server

!pip install transformers torch timm accelerate hf_xet

from . import RiCA
from .connector import transformers_adapter as tf

app = RiCA()

@app.register("sys.python.exec", True, 1000)
async def _sys_python_exec(input_, *args, **kwargs):
    """
    A tool to execute Python code.
    input:{"code": "1+1"}
    output:{"result": "2"}
    """
    try:
        code = input_.get("code", "")
        result = eval(code)
        return {"result": str(result)}
    except Exception as e:
        return {"error": str(e)}

rt = tf.ReasoningThread("")

@rt.trigger
def _output(content):
    print(content)

rt.insert("Please calculate 123*456 using `sys.python.exec` package.")
rt.wait()
print(rt.context)
rt.destroy()
